In [59]:
import pandas as pd
import requests 
from io import StringIO

requisicao = requests.get("https://pt.wikipedia.org/wiki/Campeonato_Brasileiro_de_Futebol_de_2024_-_S%C3%A9rie_A")
requisicao = requisicao.text

# tabelas = pd.read_html(StringIO(requisicao)) >>>>> premite que o programa leia StringIO como se fosse arquivo
#  pd.read_html() le somente as tabelas da requisição
tabelas = pd.read_html(StringIO(requisicao))

# for tabela in tabelas:
#     display(tabela) >>>>>> cada tabela dentro de tabelas

tabela_classificacao = tabelas[6]
# display(tabela_classificacao)
tabela_jogos = tabelas[7]
# display(tabela_jogos)






In [60]:
nome_times = list(tabela_jogos["Casa \ Fora"])
# nome dos times que estão dentro da coluna (casa/fora)

apelidos = list(tabela_jogos.columns)
# colunas da lista de jogos 

# exclui o primeiro intem das colunas 
apelidos.pop(0)

# display(nome_times)
# display(apelidos)

# faz um dicionario juntandos as duas listas 
apelido_para_time = dict(zip(apelidos, nome_times))
display(apelido_para_time)

{'ATP': 'Athletico Paranaense',
 'ATG': 'Atlético Goianiense',
 'ATM': 'Atlético Mineiro',
 'BAH': 'Bahia',
 'BOT': 'Botafogo',
 'COR': 'Corinthians',
 'CRI': 'Criciúma',
 'CRU': 'Cruzeiro',
 'CUI': 'Cuiabá',
 'FLA': 'Flamengo',
 'FLU': 'Fluminense',
 'FOR': 'Fortaleza',
 'GRE': 'Grêmio',
 'INT': 'Internacional',
 'JUV': 'Juventude',
 'PAL': 'Palmeiras',
 'RBB': 'Red Bull Bragantino',
 'SPA': 'São Paulo',
 'VAS': 'Vasco da Gama',
 'VIT': 'Vitória'}

In [61]:
# media de gols feitos dentro de casa
# media de gols feitos fora de casa
# media de gols sofridos dentro de casa
# media de gols sofridos fora de casa

# .set_index("Casa \ Fora") >>>>>> definindo o indice, ou seja, trasnforma os nomes dos times na primeira coluna, ao inves da numeros de posição
tabela_jogos_ajustada = tabela_jogos.set_index("Casa \ Fora")

# unstack pega oque esta em coluna e joga pra linha 
tabela_jogos_ajustada = tabela_jogos_ajustada.unstack().reset_index()

# renomeia os nomes das colunas 
tabela_jogos_ajustada = tabela_jogos_ajustada.rename(columns= {"level_0":"Fora", 0 :"Resultados", "Casa \ Fora": "Casa" })

def alterar_apelidos(linha):
    apelidos = linha["Fora"]
    nome = apelido_para_time[apelidos]
    return nome

#  axis= 1 >>>> linha da tabela  axis= 0 coluna da tabela
# tabela_jogos_ajustada.apply() >>>> permite voce passar uma funçao, e retorna um valor e edita linha por linha o valor recebido 
# tabela_jogos_ajustada["Fora"] >>>>>> recebe o valor retornado da .apply() 
tabela_jogos_ajustada["Fora"] = tabela_jogos_ajustada.apply(alterar_apelidos, axis= 1)
  
# tirei a possibilidade de o time enfrentar a si próprio. tabela recebe coluna fora difrente coluna casa ------ filtrei
tabela_jogos_ajustada = tabela_jogos_ajustada[ tabela_jogos_ajustada["Fora"] != tabela_jogos_ajustada["Casa"]]

display(tabela_jogos_ajustada)

,Fora,Casa,Resultados
1,Athletico Paranaense,Atlético Goianiense,1–2
2,Athletico Paranaense,Atlético Mineiro,NaN
3,Athletico Paranaense,Bahia,NaN
4,Athletico Paranaense,Botafogo,1–1
5,Athletico Paranaense,Corinthians,a
...,...,...,...
394,Vitória,Juventude,1–1
395,Vitória,Palmeiras,0–2
396,Vitória,Red Bull Bragantino,2–1
397,Vitória,São Paulo,2–1


In [62]:
tabela_jogos_ajustada["Resultados"] = tabela_jogos_ajustada["Resultados"].fillna("a jogar")
# preenche os valores vazios com "a jogar"

# variavel recebe os valores que contem o (–) 
tabela_jogos_realizados = tabela_jogos_ajustada[tabela_jogos_ajustada["Resultados"].str.contains("–")]

# variavel recebe os valores que não contem o (–) 
tabela_jogos_faltante = tabela_jogos_ajustada[~tabela_jogos_ajustada["Resultados"].str.contains("–")]

# tabela_jogos_faltante.drop(columns=["Resultados"]) >>>>> remove a coluna de resultados 
tabela_jogos_faltante = tabela_jogos_faltante.drop(columns=["Resultados"])

display(tabela_jogos_realizados)
display(tabela_jogos_faltante)

,Fora,Casa,Resultados
1,Athletico Paranaense,Atlético Goianiense,1–2
4,Athletico Paranaense,Botafogo,1–1
7,Athletico Paranaense,Cruzeiro,2–0
8,Athletico Paranaense,Cuiabá,1–2
11,Athletico Paranaense,Fortaleza,1–0
...,...,...,...
394,Vitória,Juventude,1–1
395,Vitória,Palmeiras,0–2
396,Vitória,Red Bull Bragantino,2–1
397,Vitória,São Paulo,2–1


,Fora,Casa
2,Athletico Paranaense,Atlético Mineiro
3,Athletico Paranaense,Bahia
5,Athletico Paranaense,Corinthians
6,Athletico Paranaense,Criciúma
9,Athletico Paranaense,Flamengo
...,...,...
382,Vitória,Atlético Mineiro
384,Vitória,Botafogo
386,Vitória,Criciúma
389,Vitória,Flamengo


In [63]:
# split("–") >>>> separa os valor antes e depois do (–)
# expand=True >>>> coloca o primeiro informação antes do (–) e coloca para primeira coluna e pega o segunda informação depois do (–) na segundo coluna AUTOMATICAMENTE
tabela_jogos_realizados[["gols_casa","gols_fora" ]] = tabela_jogos_realizados["Resultados"].str.split("–", expand=True)

# tirei a coluna de reultados 
tabela_jogos_realizados = tabela_jogos_realizados.drop(columns=["Resultados"])

# .astype(int) >>>> tranformando str em int 
tabela_jogos_realizados["gols_casa"] = tabela_jogos_realizados["gols_casa"].astype(int)
tabela_jogos_realizados["gols_fora"] = tabela_jogos_realizados["gols_fora"].astype(int)

display(tabela_jogos_realizados)


C:\Users\hicar\AppData\Local\Temp\ipykernel_19752\3105528355.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tabela_jogos_realizados[["gols_casa","gols_fora" ]] = tabela_jogos_realizados["Resultados"].str.split("–", expand=True)
C:\Users\hicar\AppData\Local\Temp\ipykernel_19752\3105528355.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tabela_jogos_realizados[["gols_casa","gols_fora" ]] = tabela_jogos_realizados["Resultados"].str.split("–", expand=True)


,Fora,Casa,gols_casa,gols_fora
1,Athletico Paranaense,Atlético Goianiense,1,2
4,Athletico Paranaense,Botafogo,1,1
7,Athletico Paranaense,Cruzeiro,2,0
8,Athletico Paranaense,Cuiabá,1,2
11,Athletico Paranaense,Fortaleza,1,0
...,...,...,...,...
394,Vitória,Juventude,1,1
395,Vitória,Palmeiras,0,2
396,Vitória,Red Bull Bragantino,2,1
397,Vitória,São Paulo,2,1


In [64]:

# .groupby() >>>>>> agrupa todas as linhas que contém o mesmo nome em um única linha 
# .mean(numeric_only=True) >>>> faz a média Somente de numeros 
media_gols_casa = tabela_jogos_realizados.groupby("Casa").mean(numeric_only=True)
media_gols_casa = media_gols_casa.rename(columns={"gols_casa": "gols_feitos_casa", "gols_fora": "gols_sofridos_casa"})
display(media_gols_casa)

media_gols_fora = tabela_jogos_realizados.groupby("Fora").mean(numeric_only=True)
media_gols_fora = media_gols_fora.rename(columns={"gols_casa": "gols_sofridos_fora", "gols_fora": "gols_feitos_fora"})
display(media_gols_fora)

,gols_feitos_casa,gols_sofridos_casa
Casa,,
Athletico Paranaense,1.400000,1.200000
Atlético Goianiense,0.916667,1.666667
Atlético Mineiro,1.333333,1.416667
Bahia,1.416667,0.583333
Botafogo,1.818182,0.818182
Corinthians,1.363636,1.000000
Criciúma,1.307692,1.384615
Cruzeiro,1.818182,0.636364
Cuiabá,0.818182,1.363636


,gols_sofridos_fora,gols_feitos_fora
Fora,,
Athletico Paranaense,1.166667,1.000000
Atlético Goianiense,1.416667,0.750000
Atlético Mineiro,1.400000,1.300000
Bahia,1.500000,1.333333
Botafogo,1.153846,1.615385
Corinthians,1.461538,0.384615
Criciúma,1.500000,1.400000
Cruzeiro,1.384615,0.846154
Cuiabá,1.727273,1.090909


In [65]:
# merge() >>>>> mescla as duas tabelas, ou seja, torna as duas tabelas em uma tabela só 
# left_index=True, right_index=True >>>>>>>> basicamente passa as informações 
tabela_estatisticas = media_gols_casa.merge(media_gols_fora, left_index=True, right_index=True)

# tirei o indice que era "casa", tranformei em coluna e passei para índice" 0,1,2,3,4,5..."
tabela_estatisticas = tabela_estatisticas.reset_index()

# troquei o nome da coluna casa para time 
tabela_estatisticas = tabela_estatisticas.rename(columns={"Casa": "Time"})
display(tabela_estatisticas)

,Time,gols_feitos_casa,gols_sofridos_casa,gols_sofridos_fora,gols_feitos_fora
0,Athletico Paranaense,1.400000,1.200000,1.166667,1.000000
1,Atlético Goianiense,0.916667,1.666667,1.416667,0.750000
2,Atlético Mineiro,1.333333,1.416667,1.400000,1.300000
3,Bahia,1.416667,0.583333,1.500000,1.333333
4,Botafogo,1.818182,0.818182,1.153846,1.615385
5,Corinthians,1.363636,1.000000,1.461538,0.384615
6,Criciúma,1.307692,1.384615,1.500000,1.400000
7,Cruzeiro,1.818182,0.636364,1.384615,0.846154
8,Cuiabá,0.818182,1.363636,1.727273,1.090909
9,Flamengo,1.583333,1.000000,1.272727,1.727273


In [66]:
from scipy.stats import poisson 
# scippy >>>> modulo de pacote cientificos estatisticos
# poisson da a probabilidade de resultados 

def calcular_pontuacao_esperada(linha):

    # como apartir das médias calcular a probabilidade de vitória/derrota/ empate 
    time_casa = linha["Casa"]
    time_fora = linha["Fora"]


    # média de gols do cruzeiro, dentro de casa, contra o vasco fora de casa 
    # .iloc[0] >>>>>> pega o valor de acordo com sua posição na tabela, neste caso só tem um então é [0]
    lambda_casa = tabela_estatisticas.loc[tabela_estatisticas["Time"] == time_casa, "gols_feitos_casa"].iloc[0] * tabela_estatisticas.loc[tabela_estatisticas["Time"]== time_fora, "gols_sofridos_fora"].iloc[0]
    # média de gols casa cruzeiro * média de gols sofridos do vasco  fora de casa

    # média de gols que o vasco faz fora de casa * média de gols que o cruzeiro sofre dentro de casa
    lambda_fora = tabela_estatisticas.loc[tabela_estatisticas["Time"] == time_fora, "gols_feitos_fora"].iloc[0] * tabela_estatisticas.loc[tabela_estatisticas["Time"]== time_casa, "gols_sofridos_casa"].iloc[0]



    prob_vitoria_casa = 0
    prob_vitoria_fora = 0
    prob_empate = 0

    for gols_casa in range(8):
        for gols_fora in range(8):
            # probabilidade de resultados 
        
            probabilidade_resultado = poisson.pmf(gols_casa, lambda_casa) * poisson.pmf(gols_fora, lambda_fora)
            # print(probabilidade_resultado)  # .pmf >>>> função do poisson que calcula as possibilidades

            if gols_casa == gols_fora:
                prob_empate += probabilidade_resultado
            elif gols_casa > gols_fora:
                prob_vitoria_casa += probabilidade_resultado
            elif gols_casa < gols_fora:
                prob_vitoria_fora += probabilidade_resultado

    # print(f"O {time_casa} tem {prob_vitoria_casa} de chance de ganhar ")
    # print(f"O {time_fora} tem {prob_vitoria_fora} de chance de vitória " )
    # print(f"A probabilidade de empate é de {prob_empate}")

    #  apartir da probabilidade calcular a pontuação que o time vai ter no final       
    pontuacao_casa = (prob_vitoria_casa * 3) + (prob_empate)
    pontuacao_fora = (prob_vitoria_fora * 3) + (prob_empate)
    
    # criei uma duas colunas novas 
    linha["pontos_casa"] = pontuacao_casa
    linha["pontos_fora"] = pontuacao_fora
   
    return linha
    
 

    #  apartir da probabilidade calcular a pontuação que o time vai ter no final


In [67]:
tabela_jogos_faltante = tabela_jogos_faltante.apply(calcular_pontuacao_esperada, axis=1)
display(tabela_jogos_faltante)

,Fora,Casa,pontos_casa,pontos_fora
2,Athletico Paranaense,Atlético Mineiro,1.469468,1.285884
3,Athletico Paranaense,Bahia,2.141125,0.620333
5,Athletico Paranaense,Corinthians,1.783621,0.964704
6,Athletico Paranaense,Criciúma,1.470378,1.282179
9,Athletico Paranaense,Flamengo,1.940870,0.831055
...,...,...,...,...
382,Vitória,Atlético Mineiro,1.943733,0.849523
384,Vitória,Botafogo,2.551808,0.297739
386,Vitória,Criciúma,1.940574,0.850826
389,Vitória,Flamengo,2.357151,0.472846


In [68]:
tabela_classificacao = tabela_classificacao.rename(columns={"Equipevde":"Times"})
# trnsformei a tabela de classificação em somente duas colunas 
tabela_classificacao_atualizada = tabela_classificacao[["Times", "Pts"]]



# garantido que a coluna de pts seja numeros e não string
tabela_classificacao_atualizada["Pts"] = tabela_classificacao_atualizada["Pts"].astype(int)


display(tabela_classificacao_atualizada)
# display(tabela_classificacao)




C:\Users\hicar\AppData\Local\Temp\ipykernel_19752\2293478417.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tabela_classificacao_atualizada["Pts"] = tabela_classificacao_atualizada["Pts"].astype(int)


,Times,Pts
0,Fortaleza,48
1,Botafogo,47
2,Palmeiras,44
3,Flamengo,44
4,São Paulo,41
5,Bahia,39
6,Cruzeiro,38
7,Vasco da Gama,31
8,Atlético Mineiro,30
9,Athletico Paranaense,29


In [69]:
# pontuação de cada time feitos em casa 
tabela_pontuacao_casa = tabela_jogos_faltante.groupby("Casa").sum(numeric_only =True)[["pontos_casa"]]
tabela_pontuacao_fora = tabela_jogos_faltante.groupby("Fora").sum(numeric_only =True)[["pontos_fora"]]
display(tabela_pontuacao_fora)
display(tabela_pontuacao_casa)

,pontos_fora
Fora,
Athletico Paranaense,6.889526
Atlético Goianiense,4.430320
Atlético Mineiro,8.673770
Bahia,6.149922
Botafogo,8.832618
Corinthians,2.874036
Criciúma,8.709795
Cruzeiro,4.983734
Cuiabá,5.638665


,pontos_casa
Casa,
Athletico Paranaense,16.175292
Atlético Goianiense,7.923910
Atlético Mineiro,11.198365
Bahia,14.863405
Botafogo,19.488701
Corinthians,14.542375
Criciúma,10.032684
Cruzeiro,18.958877
Cuiabá,9.996077


In [70]:
# função soma os pontos de jogos realizados + jogos faltantes dentro e fora de casa 
def atualizar_pontucao_precisao(linha):
    time = linha["Times"]

    pontuacao = int(linha["Pts"]) + float(tabela_pontuacao_casa.loc[time, "pontos_casa"] ) + float(tabela_pontuacao_fora.loc[time, "pontos_fora"])
    return pontuacao

tabela_classificacao_atualizada["Pts"] = tabela_classificacao_atualizada.apply(atualizar_pontucao_precisao, axis=1)

# atualizei os índices colanco + 1, pois, quando se trata de classificação não existe posição 0( obs:toda vez qi=ue rodar ele adiciona 1 em cada indice)

# sort_values(by="Pts", ascending=False)  >>>>>> coloquei em uma ordem decrescente(classificação é descrescente )
# se  fosse sort_values(by="Pts", ascending=True) >>>> iria ser crescente 
tabela_classificacao_atualizada = tabela_classificacao_atualizada.sort_values(by="Pts", ascending=False)
tabela_classificacao_atualizada = tabela_classificacao_atualizada.reset_index(drop=True)
tabela_classificacao_atualizada.index =  tabela_classificacao_atualizada.index + 1

display(tabela_classificacao_atualizada)




C:\Users\hicar\AppData\Local\Temp\ipykernel_19752\3518255980.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tabela_classificacao_atualizada["Pts"] = tabela_classificacao_atualizada.apply(atualizar_pontucao_precisao, axis=1)


,Times,Pts
1,Botafogo,75.321319
2,Fortaleza,70.485936
3,Flamengo,70.023541
4,Palmeiras,68.214311
5,São Paulo,63.239913
6,Cruzeiro,61.942610
7,Bahia,60.013326
8,Internacional,52.286983
9,Athletico Paranaense,52.064818
10,Juventude,50.088937
